In [182]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets
import torchvision.transforms as T
from torchvision.utils import make_grid, save_image

import time
from PIL import Image

from tqdm import tqdm
from matplotlib import pyplot as plt
%matplotlib inline

is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

to_pil_image = T.ToPILImage()

GPU not available, CPU used


In [228]:
mnist_train = datasets.MNIST(
    root='data',
    train=True,
    download=True,
    transform=T.Compose([T.Resize(32), T.ToTensor(), T.Lambda(lambda x: torch.flatten(x))])
)

In [229]:
svhn_train = datasets.SVHN(
    root='data',
    download=True,
    transform=T.Compose([ T.ToTensor(), T.Lambda(lambda x: torch.flatten(x))])
)

Using downloaded and verified file: data/train_32x32.mat


In [198]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size = 300, latent_size = 100):
        super(Encoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.LeakyReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.LeakyReLU(),
            nn.Linear(hidden_size, 2 * latent_size)
        )
    
    def sample(self, mu, sigma):
        eps = torch.randn_like(sigma)
        return mu + sigma * eps
        
    def forward(self, x):
        h = self.encoder(x)

        mu, log_var = torch.chunk(h, 2, dim=1)
        sigma = torch.exp(0.5*log_var) 
        z = self.sample(mu, sigma)
        
        return z, mu, sigma

class Decoder(nn.Module):
    def __init__(self, output_size, hidden_size = 300, latent_size = 100):
        super(Decoder, self).__init__()
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_size, hidden_size),
            nn.LeakyReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.LeakyReLU(),
            nn.Linear(hidden_size, output_size),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.decoder(x) 
    
class Prior(nn.Module):
    def __init__(self, batch_size, latent_size=100):
        super(Prior, self).__init__()
        self.latent_size = latent_size
        self.p = torch.distributions.Normal(
            torch.zeros((batch_size, self.latent_size)),
            torch.ones((batch_size, self.latent_size))
        )

    def sample(self, batch_size):
        z = torch.randn((batch_size, self.latent_size))
        return z

    def log_prob(self, z):

        return self.p.log_prob(z)
        

class VAE(nn.Module):
    def __init__(self, input_size):
        super(VAE, self).__init__()
        self.encoder = Encoder(input_size)
        self.decoder = Decoder(input_size)
        
    def forward(self, x):
        z, mu, sigma = self.encoder(x)
                
        return self.decoder(z), z, mu, sigma
    
    
class ELBO():
    def __init__(self, prior):
        self.prior = prior
        self.reconstruction_error = nn.BCELoss(reduction='none')
    
    def kullback_Leibler_divergence(self, z, mu, sigma):
        q = torch.distributions.Normal(mu, sigma)

        log_qz = q.log_prob(z)
        log_pz = self.prior.log_prob(z)
        
        kl = (log_qz - log_pz).sum(-1)
        
        return kl
    
    def __call__(self, inputs, outputs, z, mu, sigma):
        
        re = self.reconstruction_error(outputs, inputs).sum(-1)
        kl = self.kullback_Leibler_divergence(z, mu, sigma)

        elbo = re + kl
        return elbo.mean()

In [230]:

def train(net, train_data, img_dim, batch_size=10, learning_rate=0.0001, epochs=20, nr_test_samples=64):
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    prior = Prior(batch_size)
    criterion = ELBO(prior)
    
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
    running_loss = []

    for epoch in range(epochs):
        print("Epoch:" , epoch+1)

        for i, data in tqdm(enumerate(train_loader), total=int(len(train_data)/train_loader.batch_size)):
            optimizer.zero_grad()

            inputs, _ = data
            inputs = inputs.to(device)

            # Forward
            outputs, z, mu, sigma = net(inputs)
            
            # Backward
            loss = criterion(inputs, outputs, z, mu, sigma)
            loss.backward()
            optimizer.step()
            
            running_loss.append(loss.item())
            
        sample = prior.sample(nr_test_samples)
        generated_img = net.decoder(sample).view(nr_test_samples,img_dim,32,32)
        generated_img = make_grid(generated_img)
    
        #SAVE IMAGE
        im = Image.fromarray(np.array(to_pil_image(generated_img)))
        im.save(f"Images_output/vae/epoch_{epoch}.jpeg")

        print(f'Loss: {np.mean(running_loss[-len(train_data):])}')
    
    return running_loss




In [ ]:
input_size = train_data[0][0].shape[0]

net = VAE(input_size)

loss = train(net, mnist_train, 1)


  0%|          | 11/6000 [00:00<00:56, 105.54it/s]

Epoch: 1


  0%|          | 12/6000 [00:00<00:53, 112.22it/s]

Loss: 250.30866422017417
Epoch: 2


  3%|▎         | 168/6000 [00:01<00:50, 114.37it/s]